In [18]:
import requests
import psycopg2
import os

from bs4 import BeautifulSoup
from collections import OrderedDict
from openai import OpenAI
from dotenv import load_dotenv

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()

False

In [30]:
class Page:
    def __init__(self, url):
        self.url = url
        self.title = None
        self.page_source = None
        self.content = None
        self.embeddings = None

    def parse_page(self):  # Add "self" parameter here
        try:
            options = webdriver.ChromeOptions()
            
            # If you want Chrome to run headless (without opening a window)
            options.add_argument('--headless')
            
            driver = webdriver.Chrome(options=options)

            # Navigate to the page
            driver.get(self.url)
                        
            # Now that the page is loaded, you can extract its text
            self.content = driver.find_element(By.CLASS_NAME, "publication-view").text
    
            page_source = driver.find_element(By.CLASS_NAME, "publication-view").get_attribute('outerHTML')
            self.page_source = page_source
            if page_source:
                soup = BeautifulSoup(page_source, 'html.parser')
                self.title = soup.title.string if soup.title else None
            
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            # Make sure to quit the driver to free up resources
            driver.quit()

In [31]:
# Example
main_page_url = 'https://new.express.adobe.com/webpage/vnOKwPijAc0by/?page-mode=static'
page = Page(main_page_url)
page.parse_page()
print(page.content)
print(page.title)
print(page.page_source)

Official Guides for Global Shapers
Welcome!
Founded by Klaus Schwab in 2011, the World Economic Forum's Global Shapers Community is a scalable solution for organizing and mobilizing the full potential of young people.
Over the last decade, Global Shapers have contributed knowledge, resources and tools to help members and our hubs create change. Explore the pages below to learn more about our community and how we do our work. Read our monthly newsletter for the latest updates.
To access all of our guides in another language, simply download the Google Translate browser extension.
Our Charter outlines the global standards that all members and hubs in our organization are responsible to uphold. Our Charter was created by Shapers for Shapers and includes best practices that have been identified by members since our community was created in 2011. By joining the Global Shapers Community, YOU agree to live up to its principles and values.
Meet the Global Shapers Team: Get to know your Communi

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="utf-8">\n    <title>Global Shapers Innovation Prize</title>\n    <meta content="yes" name="apple-touch-fullscreen" />\n    <meta content="yes" name="apple-mobile-web-app-capable" />\n    <meta content="black-translucent" name="apple-mobile-web-app-status-bar-style" />\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n\n    <meta name="robots" content="noindex">\n\n    <meta property="og:title" content="Global Shapers Innovation Prize">\n    <meta property="og:type" content="website">\n    <meta property="og:image" content="https://new.express.adobe.com/webpage/E7YzuCMRZ2cn1/resources/1710234701165?asset_id=rendition">\n    <meta property="og:url" content="https://new.express.adobe.com/webpage/E7YzuCMRZ2cn1">\n    <meta property="og:image:width" content="1024">\n    <meta property="og:image:height" content="512">\n    <meta property="og:site_name" content="Adobe Express">\n    <meta property="og:desc

In [ ]:
# Use BeautifulSoup to parse the HTML content
soup = BeautifulSoup(main_page_html_content, 'html.parser')
urls = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('https://digilaw.ch/')]
urls = list(OrderedDict.fromkeys(urls))

# remove specific urls
urls = [url for url in urls if not url.endswith('.pdf')]
urls = [url for url in urls if not url.endswith('15-impressum/')]
# urls = urls[:5]

url_to_filename = {url: url.split('/')[-2] + '.xhtml' for url in urls}

In [ ]:
# Create a list to store the Page objects
pages = []

# Iterate over the URLs and create Page objects
for url in urls:
    page = Page(url)
    page.parse_page()
    pages.append(page)

In [ ]:
client = OpenAI()

for page in pages:
    page.embeddings = client.embeddings.create(input = [page.content], model="text-embedding-3-small").data[0].embedding

In [ ]:
# Connect to the database
conn = psycopg2.connect(
        dbname=os.getenv("POSTGRES_DATABASE"),
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        host=os.getenv("POSTGRES_HOST")
    )

# Create a cursor object
cur = conn.cursor()

In [ ]:
# enable vector extension
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
conn.commit()

In [ ]:
# Drop the table if it exists
cur.execute('DROP TABLE IF EXISTS pages')

# Create a table to store the pages
cur.execute('''
    CREATE TABLE IF NOT EXISTS pages (
        id SERIAL PRIMARY KEY,
        url TEXT,
        title TEXT,
        content TEXT,
        embeddings vector(1536)
    )
''')

# Insert the pages into the database
for page in pages:
    cur.execute('''
        INSERT INTO pages (url, title, content, embeddings)
        VALUES (%s, %s, %s, %s)
    ''', (page.url, page.title, page.content, page.embeddings))

# Commit the transaction
conn.commit()

# Close the cursor and the connection
cur.close()

conn.close()

In [ ]:
question = "What is the legal basis for the processing of personal data?"
question_embedding = client.embeddings.create(input=[question], model="text-embedding-3-small").data[0].embedding
print(question_embedding)


[-0.022289831191301346, 0.04408017173409462, 0.0644344687461853, 0.005749361123889685, 0.028554294258356094, -0.003254502546042204, -0.033070534467697144, 0.01393374614417553, -0.02749287337064743, -0.013038822449743748, 0.06256137788295746, 0.012310396879911423, -0.004500631242990494, -0.030968504026532173, -0.0036057080142199993, -0.015453034080564976, 0.013975370675325394, 0.037940580397844315, -0.008538191206753254, -0.038294386118650436, 0.025931959971785545, 0.03798220679163933, -0.027888303622603416, -0.004596887622028589, -0.023663433268666267, -0.07146897912025452, 0.010083494707942009, -0.016982728615403175, -0.03637966886162758, 0.006613065954297781, 0.010426895692944527, -0.01016154047101736, -0.009953418746590614, 0.013631969690322876, 0.0004991668392904103, -0.007617253344506025, -0.02865835465490818, -0.0019251254852861166, 0.029927896335721016, 0.036691851913928986, 0.007513192482292652, -0.09740094095468521, 0.002112434944137931, 0.06709842383861542, -0.031572058796882

In [ ]:
# Query the database
conn = psycopg2.connect(
        dbname=os.getenv("POSTGRES_DATABASE"),
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        host=os.getenv("POSTGRES_HOST")
    )

cur = conn.cursor()
cur.execute('''
    SELECT url, title, content, embeddings
    FROM pages
    ORDER BY pages.embeddings <-> %s::vector 
    LIMIT 5
''', (question_embedding,))

results = cur.fetchall()

for result in results:
    print(result)

('https://digilaw.ch/', 'digilaw.ch – Die digitale Welt und ihre rechtlichen Hotspots', '\n\n\nDie digitale Welt und ihre rechtlichen Hotspots \n\nFür die «Digital Natives», also die Leute, die mit oder sogar im Internet aufgewachsen sind, ist die Welt heute nicht mehr analog, sondern digital. Zu diesen Digital Natives gehören meine Studierenden an der Hochschule Luzern in den Bachelor- und Masterstudiengängen und der Weiterbildung, die ich in den rechtlichen Aspekten der digitalen Welt unterrichte. Dafür habe ich ein Lehrbuch gesucht und nicht gefunden. Aus diesem Grund publiziere ich nun selbst dazu. Und klar, bei diesem Thema schreibe ich natürlich nicht ein Buch, sondern «open access» direkt im Internet. Paul Arden, Autor und früherer Creative Director bei Saatchi & Saatchi, einer international tätigen Kommunikations- und Werbeagentur, meint dazu: «Give away everything you know, and more will come back to you». Die Publikation ist ein «Work in Progress» und wird laufend aktualisier